# K-Nearest Neighbors Algorithm

In [16]:
import os
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import warnings
warnings.filterwarnings('ignore')

## Loading Dataset

In [6]:
# Get the current directory
current_dir = os.getcwd()

# Get the path to the parent directory
parent_dir = os.path.dirname(current_dir)

# Specify the filename
filename = 'final.sqlite'

# Construct the path to the file in the parent directory
file_path = os.path.join(parent_dir, filename)

# Check if the file exists
if os.path.isfile(file_path):
    conn = sqlite3.connect(file_path)
    final = pd.read_sql_query('select * from reviews', conn)
    conn.close()
else:
    print('Please run Text Preprocessing code file')

In [7]:
final.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought sever vital can dog food product found ...
1,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arriv label jumbo salt peanut peanut a...
2,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confect around centuri light pillowi citrus ge...
3,3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,look secret ingredi robitussin believ found go...
4,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffi great price wide assort yummi taff...


In [11]:
# Sorting data based on time for time based splitting
final = final.sort_values('Time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# Now we will perform time based slicing
x_train_raw = final['CleanedText'][0:50000]
y_train_raw = final['CleanedText'][0:50000]
x_cv_raw = final['CleanedText'][50000:70000]
y_cv = final['CleanedText'][50000:70000]
x_test_raw = final['CleanedText'][70000:87773]
y_test = final['CleanedText'][70000:87773]

In [13]:
# Some common initializations used in whole of the assignment
neighbors = np.arange(1,30,1)
mask = neighbors%2 != 0
neighbors = neighbors[mask]

## Applying KNN-brute force using BOW

### Training

In [18]:
count_vect = CountVectorizer()
count_vect.fit(x_train_raw.values)
filename = 'brute_bow'
pickle.dump(count_vect, open(filename, 'wb'))